# Optimizing the Linear Model with TensorBoard

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Loading TensorBoard
%load_ext tensorboard

In [3]:
from tensorboard.plugins.hparams import api as hp

In [4]:
import random
random.seed(123)

In [5]:
# import window_generator as WindowGenerator
from window_generator import WindowGenerator

# Configuring hyperparameters

In [6]:
# Optimizers (SGD, Adam, Adamax, RMSprop)
optimizers = [
    'adam', 
    'sgd', 
    'adamax', 
    'rmsprop'
]


# Loss functions (MeanSquaredError, Huber, MeanSquaredLogarithmicError)
loss_functions = [
    'huber', 
    'MeanSquaredError', 
    'MeanSquaredLogarithmicError'
]

In [7]:
# Hyperparameters
OPTIMIZER = hp.HParam('Optimizer', hp.Discrete(optimizers))
LOSS_FUNCTION = hp.HParam('Loss_function', hp.Discrete(loss_functions))
WINDOW_SIZE = hp.HParam('WINDOW_SIZE', hp.Discrete([12, 24, 48]))

# Metrics
METRIC_RMSE = 'RootMeanSquaredError'


with tf.summary.create_file_writer('logs/Linear').as_default():
    hp.hparams_config(
    # Parameters
    hparams = [
        OPTIMIZER,
        WINDOW_SIZE,
        LOSS_FUNCTION    
    ],
    # Metrics
    metrics = [
        hp.Metric(
            METRIC_RMSE, display_name='RMSE'
        ),
    ]
    )

# Loading the dataset 

In [8]:
df = pd.read_csv('extracted_data/csv/hour_merged.csv',
                 parse_dates={'data': ['date', 'time']},
                 infer_datetime_format=True,
                 dayfirst=False,
                 index_col='data'
                )


df.head()

,price,demand,emissions
data,,,
2017-01-01 00:00:00,58.82,24682,14467.2840
2017-01-01 01:00:00,58.23,24046,14173.8295
2017-01-01 02:00:00,51.95,22665,13198.0690
2017-01-01 03:00:00,47.27,21200,12510.7595
2017-01-01 04:00:00,45.49,20056,12203.7315


In [9]:
# Select data from 2021
df = df[int((len(df)*0.8)):]

# Differencing

In [10]:
df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,50.87,25134,6356.6480
2021-01-01 01:00:00,48.19,24179,6269.1480
2021-01-01 02:00:00,44.68,22472,5702.4820
2021-01-01 03:00:00,38.50,20319,5284.9275
2021-01-01 04:00:00,36.80,19040,5218.2340


In [11]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [12]:
diff_price = difference(df['price'])
diff_emissions = difference(df['emissions'])
diff_demand = difference(df['demand'])

In [13]:
s1 = pd.Series([0.0])
diff_price = s1.append(diff_price, ignore_index=True)
diff_emissions = s1.append(diff_emissions, ignore_index=True)
diff_demand = s1.append(diff_demand, ignore_index=True)

In [14]:
df['price'] = diff_price.values
df['demand'] = diff_demand.values
df['emissions'] = diff_emissions.values

In [15]:
df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,0.00,0.0,0.0000
2021-01-01 01:00:00,-2.68,-955.0,-87.5000
2021-01-01 02:00:00,-3.51,-1707.0,-566.6660
2021-01-01 03:00:00,-6.18,-2153.0,-417.5545
2021-01-01 04:00:00,-1.70,-1279.0,-66.6935


# Split into training and test

In [16]:
# 80% for training and 20% for test
n = len(df)
train_df = df[0:int(n*0.8)]
test_df = df[int(n*0.8):]

# Standarization

In [17]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [18]:
train_df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,-0.001166,0.000057,-0.000507
2021-01-01 01:00:00,-0.121941,-0.737243,-0.163255
2021-01-01 02:00:00,-0.159345,-1.317819,-1.054494
2021-01-01 03:00:00,-0.279669,-1.662149,-0.777150
2021-01-01 04:00:00,-0.077777,-0.987385,-0.124555


# Linear Model

In [19]:
MAX_EPOCHS = 20
patience = 4
OUT_STEPS = 24
num_features=df.shape[1] 

In [20]:
multi_val_performance = {}
multi_performance = {}

In [21]:
def train_test_model(hparams):
    multi_linear_model = tf.keras.Sequential([
        # Seleccionar el último timestep
        # Forma: [batch, time, features] => [batch, 1, features]
        tf.keras.layers.Lambda(lambda x: x[:, -1, :]),
        # Shape => [batch, 1, out_steps*features]
        tf.keras.layers.Dense(OUT_STEPS*num_features,
                             kernel_initializer=tf.initializers.zeros()),
        # Shape =>[batch, out_steps, features]
        tf.keras.layers.Reshape([OUT_STEPS, num_features])
    ])
    
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    multi_linear_model.compile(loss=hparams[LOSS_FUNCTION],
                 optimizer=hparams[OPTIMIZER],
                 metrics=[tf.metrics.RootMeanSquaredError()])
    
    # Creating a Window
    window = WindowGenerator(input_width=hparams[WINDOW_SIZE], label_width=OUT_STEPS, shift=OUT_STEPS, train_df=train_df, test_df=test_df)
    
    # History
    history = multi_linear_model.fit(window.train, epochs=MAX_EPOCHS, validation_data = window.test, callbacks=[early_stopping])
    
    print(multi_linear_model.evaluate(window.test, verbose=0))
    loss, rmse = multi_linear_model.evaluate(window.test, verbose=0)
    
    return rmse
    

In [22]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        rmse = train_test_model(hparams)
        
        tf.summary.scalar(METRIC_RMSE, rmse, step=1)

In [23]:
session_num = 0

for optimizer in OPTIMIZER.domain.values:
    for window_size in WINDOW_SIZE.domain.values:
        for loss_function in LOSS_FUNCTION.domain.values:
            hparams = {
                OPTIMIZER: optimizer,
                WINDOW_SIZE: window_size,
                LOSS_FUNCTION: loss_function
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/Linear/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'Optimizer': 'adam', 'WINDOW_SIZE': 12, 'Loss_function': 'MeanSquaredError'}
Epoch 1/20
218/218 [==============================] - 1s 2ms/step - loss: 0.9403 - root_mean_squared_error: 0.9697 - val_loss: 1.3245 - val_root_mean_squared_error: 1.1509
Epoch 2/20
218/218 [==============================] - 0s 2ms/step - loss: 0.8885 - root_mean_squared_error: 0.9426 - val_loss: 1.2878 - val_root_mean_squared_error: 1.1348
Epoch 3/20
218/218 [==============================] - 0s 2ms/step - loss: 0.8718 - root_mean_squared_error: 0.9337 - val_loss: 1.2775 - val_root_mean_squared_error: 1.1303
Epoch 4/20
218/218 [==============================] - 0s 2ms/step - loss: 0.8646 - root_mean_squared_error: 0.9299 - val_loss: 1.2739 - val_root_mean_squared_error: 1.1287
Epoch 5/20
218/218 [==============================] - 0s 2ms/step - loss: 0.8613 - root_mean_squared_error: 0.9281 - val_loss: 1.2730 - val_root_mean_squared_error: 1.1283
Epoch 6/20
218/218 [================

Epoch 2/20
218/218 [==============================] - 0s 2ms/step - loss: 0.3157 - root_mean_squared_error: 0.9452 - val_loss: 0.4445 - val_root_mean_squared_error: 1.1383
Epoch 3/20
218/218 [==============================] - 0s 2ms/step - loss: 0.3112 - root_mean_squared_error: 0.9369 - val_loss: 0.4422 - val_root_mean_squared_error: 1.1335
Epoch 4/20
218/218 [==============================] - 0s 2ms/step - loss: 0.3093 - root_mean_squared_error: 0.9333 - val_loss: 0.4417 - val_root_mean_squared_error: 1.1321
Epoch 5/20
218/218 [==============================] - 1s 2ms/step - loss: 0.3084 - root_mean_squared_error: 0.9317 - val_loss: 0.4415 - val_root_mean_squared_error: 1.1316
Epoch 6/20
218/218 [==============================] - 1s 2ms/step - loss: 0.3080 - root_mean_squared_error: 0.9309 - val_loss: 0.4415 - val_root_mean_squared_error: 1.1316
Epoch 7/20
218/218 [==============================] - 1s 2ms/step - loss: 0.3078 - root_mean_squared_error: 0.9306 - val_loss: 0.4418 - val_

--- Starting trial: run-14
{'Optimizer': 'adamax', 'WINDOW_SIZE': 24, 'Loss_function': 'huber'}
Epoch 1/20
218/218 [==============================] - 1s 3ms/step - loss: 0.3369 - root_mean_squared_error: 0.9816 - val_loss: 0.4663 - val_root_mean_squared_error: 1.1765
Epoch 2/20
218/218 [==============================] - 0s 2ms/step - loss: 0.3235 - root_mean_squared_error: 0.9600 - val_loss: 0.4530 - val_root_mean_squared_error: 1.1550
Epoch 3/20
218/218 [==============================] - 1s 2ms/step - loss: 0.3169 - root_mean_squared_error: 0.9488 - val_loss: 0.4471 - val_root_mean_squared_error: 1.1444
Epoch 4/20
218/218 [==============================] - 0s 2ms/step - loss: 0.3135 - root_mean_squared_error: 0.9425 - val_loss: 0.4443 - val_root_mean_squared_error: 1.1388
Epoch 5/20
218/218 [==============================] - 0s 2ms/step - loss: 0.3114 - root_mean_squared_error: 0.9386 - val_loss: 0.4428 - val_root_mean_squared_error: 1.1357
Epoch 6/20
218/218 [========================

218/218 [==============================] - 0s 2ms/step - loss: 0.1596 - root_mean_squared_error: 1.0009 - val_loss: 0.2201 - val_root_mean_squared_error: 1.2275
Epoch 3/20
218/218 [==============================] - 0s 2ms/step - loss: 0.1596 - root_mean_squared_error: 1.0009 - val_loss: 0.2201 - val_root_mean_squared_error: 1.2275
Epoch 4/20
218/218 [==============================] - 0s 2ms/step - loss: 0.1596 - root_mean_squared_error: 1.0009 - val_loss: 0.2201 - val_root_mean_squared_error: 1.2275
Epoch 5/20
218/218 [==============================] - 1s 2ms/step - loss: 0.1596 - root_mean_squared_error: 1.0009 - val_loss: 0.2201 - val_root_mean_squared_error: 1.2275
Epoch 6/20
218/218 [==============================] - 0s 2ms/step - loss: 0.1596 - root_mean_squared_error: 1.0009 - val_loss: 0.2201 - val_root_mean_squared_error: 1.2275
Epoch 7/20
218/218 [==============================] - 0s 2ms/step - loss: 0.1596 - root_mean_squared_error: 1.0009 - val_loss: 0.2201 - val_root_mean_s

218/218 [==============================] - 1s 2ms/step - loss: 0.9922 - root_mean_squared_error: 0.9961 - val_loss: 1.4733 - val_root_mean_squared_error: 1.2138
Epoch 2/20
218/218 [==============================] - 0s 2ms/step - loss: 0.9747 - root_mean_squared_error: 0.9873 - val_loss: 1.4452 - val_root_mean_squared_error: 1.2021
Epoch 3/20
218/218 [==============================] - 0s 2ms/step - loss: 0.9599 - root_mean_squared_error: 0.9797 - val_loss: 1.4215 - val_root_mean_squared_error: 1.1923
Epoch 4/20
218/218 [==============================] - 0s 2ms/step - loss: 0.9472 - root_mean_squared_error: 0.9733 - val_loss: 1.4016 - val_root_mean_squared_error: 1.1839
Epoch 5/20
218/218 [==============================] - 0s 2ms/step - loss: 0.9364 - root_mean_squared_error: 0.9677 - val_loss: 1.3847 - val_root_mean_squared_error: 1.1767
Epoch 6/20
218/218 [==============================] - 1s 2ms/step - loss: 0.9272 - root_mean_squared_error: 0.9629 - val_loss: 1.3704 - val_root_mean_s

[0.4613649845123291, 1.1686525344848633]
--- Starting trial: run-33
{'Optimizer': 'sgd', 'WINDOW_SIZE': 48, 'Loss_function': 'MeanSquaredError'}
Epoch 1/20
217/217 [==============================] - 1s 2ms/step - loss: 0.9934 - root_mean_squared_error: 0.9967 - val_loss: 1.4770 - val_root_mean_squared_error: 1.2153
Epoch 2/20
217/217 [==============================] - 0s 2ms/step - loss: 0.9760 - root_mean_squared_error: 0.9879 - val_loss: 1.4489 - val_root_mean_squared_error: 1.2037
Epoch 3/20
217/217 [==============================] - 0s 2ms/step - loss: 0.9612 - root_mean_squared_error: 0.9804 - val_loss: 1.4252 - val_root_mean_squared_error: 1.1938
Epoch 4/20
217/217 [==============================] - 0s 2ms/step - loss: 0.9486 - root_mean_squared_error: 0.9740 - val_loss: 1.4053 - val_root_mean_squared_error: 1.1854
Epoch 5/20
217/217 [==============================] - 0s 2ms/step - loss: 0.9378 - root_mean_squared_error: 0.9684 - val_loss: 1.3884 - val_root_mean_squared_error: 1.

Epoch 20/20
217/217 [==============================] - 0s 2ms/step - loss: 0.3248 - root_mean_squared_error: 0.9629 - val_loss: 0.4614 - val_root_mean_squared_error: 1.1702
[0.4613906443119049, 1.1702266931533813]
